In [ ]:
import torch

major_version, minor_version = torch.cuda.get_device_capability()

%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 256
dtype = None
load_in_4bit = False
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="HuggingFaceH4/zephyr-7b-beta",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    #token = ,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

HuggingFaceH4/zephyr-7b-beta does not have a padding token! Will use pad_token = <unk>.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,  # 드롭아웃
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # 타겟 모듈
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=123,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset

EOS_TOKEN = tokenizer.eos_token
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.


### Instruction:
{}

### Response:
{}"""
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

dataset = load_dataset("mintaeng/llm_futsaldata_yo", split="train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Generating train split:   0%|          | 0/23480 [00:00<?, ? examples/s]

Map:   0%|          | 0/23480 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=20,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=3,
        max_steps = 1761,
        logging_steps = 10,
        learning_rate=2e-5,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=123,
        output_dir="outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/23480 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 23,480 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 20 | Gradient Accumulation steps = 2
\        /    Total batch size = 40 | Total steps = 1,761
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
10,1.681700
20,1.427600
30,1.371600
40,1.350400
50,1.294200
60,1.286400
70,1.278000
80,1.285800
90,1.249700
100,1.243800


Step,Training Loss
10,1.681700
20,1.427600
30,1.371600
40,1.350400
50,1.294200
60,1.286400
70,1.278000
80,1.285800
90,1.249700
100,1.243800


In [ ]:
trainer_stats

TrainOutput(global_step=1761, training_loss=1.1261051157399513, metrics={'train_runtime': 26645.6613, 'train_samples_per_second': 2.644, 'train_steps_per_second': 0.066, 'total_flos': 7.784199669311078e+17, 'train_loss': 1.1261051157399513, 'epoch': 3.0})

In [ ]:

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

26645.6613 seconds used for training.
444.09 minutes used for training.
Peak reserved memory = 17.852 GB.
Peak reserved memory for training = 3.463 GB.
Peak reserved memory % of max memory = 80.53 %.
Peak reserved memory for training % of max memory = 15.622 %.


In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id
    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )
stop_token = "<|end_of_text|>"
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[0]

stopping_criteria = StoppingCriteriaList([StopOnToken(stop_token_id)])

(예시)

In [ ]:
from transformers import TextStreamer
inputs = tokenizer(
    [
        prompt.format(
            "제시하는 context안에서만 대답해.\
            \ncontext : [Document(page_content='146해석과권장사항규칙3 –선수들주심들의허가로피치밖에있는선수가,교체를하지않고,주심들또는3주심으로부터허가없이피치에재입장하여, 또다른경고성반칙을범했다면, 두개의경고성반칙에의해그선수를퇴장시켜야한다,예를들어, 주심들중한명의허가없이입장하여무모한태도로상대편을거는경우.이러한반칙이과도한힘으로범했다면, 그선수는직접퇴장이다.선수가피치의경계선중하나를우연히넘어가거나, 플레이움직임의일부로피치를떠난선수는반칙을범했다고간주되지않는다. 교체선수교체선수가교체절차를위반하여피치에입장하거나또는그팀의추가선수로플레이를하게되었다면, 주심들은주심외심판들의도움으로,다음의지침에따라야한다:\n•플레이중단, 어드밴티지를적용할수있다면, 즉시중지시킬필요는없다.\n•그팀의추가선수로플레이를한다면반스포츠적행동또는올바르지않게교체가이루어졌다면교체절차위반에의해경고를주어야한다.\n•교체절차를완료하지않고입장하였거나또는그의팀의추가선수로플레이한이유의반칙이었고,그들이기술지역으로이동하기전이었다면,피치를떠나지않은교체선수는다음중단시에피치를떠나야한다.\n•주심들이어드밴티지를적용했다면:  \n•교체한팀이볼을소유했을때플레이를중단하여야하고볼이페널티에어리어안에있을때플레이가중단된경우를제외하고플레이가중단됐을때볼이있던위치에서상대팀의간접프리킥으로재개한다. (참고, 규칙13)\n•교체한팀이간접프리킥,직접프리킥또는페널티킥으로처벌되는반칙또는교체선수가플레이를방해한경우, 상대팀에게관련된재개를주어교체선수팀을처벌하여야한다   필요하다면, 주심들역시범해진반칙에상응하는징계조치를한다.\n•교체한팀의상대팀이반칙을범하여볼이아웃오브플레이가되는이유로플레이를중단한다음,   교체한팀의상대팀에게간접프리킥으로재개하여야한다.   필요하다면, 주심들역시범해진반칙에상응하는징계조치를한다.', metadata={'author': 'na', 'creationDate': 'D:20240103170645+09'00', 'creator': 'Microsoft® Word Microsoft 365용', 'file_path': '23-24풋살경기규칙.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': D:20240103170645+09'00', 'page': 145, 'producer': 'Microsoft® Word Microsoft 365용', 'source': '23-24풋살경기규칙.pdf', 'subject': '', 'title': '', 'total_pages': 175, 'trapped': ''}),\nDocument(page_content='•피치안에;\n•페널티마크에서최소5m\n•페널티마크뒤;\n•페널티에어리어밖.선수들이규칙에따라위치를잡은후, 주심들중한명이페널티킥을실시하기위한신호를한다페널티킥을실시하는선수는볼을앞으로킥해야한다. 볼이앞으로이동한다면발뒤꿈치로차는것(백힐링)은허용된다.볼이킥이되는순간, 수비팀골키퍼의한발은그일부라도골라인에닿아있거나그선과동일선상에위치해야한다.볼이킥이되어명백하게이동했을때인플레이가된다.키커는다른선수가볼을터치하기전까지는다시볼을플레이하면안된다.전·후반시간의종료시점에페널티킥이주어진다면, 그시간은페널티킥이완료될때끝난것으로간주된다.볼이인플레이된후, 다음중하나가발생하면그킥이완료된것으로간주된다:\n•볼의움직임이멈추었거나또는피치밖으로벗어났을때;\n•수비측골키퍼가아닌다른선수(키커포함)가볼을플레이한경우;\n•주심이키커또는키커의팀동료의반칙으로플레이를중단한경우.', metadata={'author': 'na', 'creationDate': D:20240103170645+09'00', 'creator': 'Microsoft® Word Microsoft 365용', 'file_path': '23-24풋살경기규칙.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': D:20240103170645+09'00', 'page': 93, 'producer': 'Microsoft® Word Microsoft 365용', 'source': '23-24풋살경기규칙.pdf', 'subject': '', 'title': '', 'total_pages': 175, 'trapped': ''}),\nDocument(page_content='34           11    피치밖선수의부적절한재입장선수가피치에다시들어올때주심들중한명의허락이필요하지만, 주심의허락없이들어왔다면주심들은반드시:\n•플레이를중단한다(선수가플레이방해또는심판진을방해하지않았거나, 어드밴티지를적용할수있다면즉시중단시키지않는다).\n•허락없이피치에들어온이유로선수에게경고조치한다.주심들이플레이를중단시켰다면, 다음과같이재개한다:\n•방해지점에서직접프리킥또는페널티에어리어내에서방해라면페널티킥으로재개하고파울은누적된다.\n•방해가없었을경우, 중단되었을때볼이있었던지점에서간접프리킥으로재개.선수가플레이하는움직임중에경계선을넘어갔다면, 반칙을범한것으로간주하지않는다.12    팀주장팀의주장은특별한지위나특권을갖고있지는않지만, 팀의행동에대해어느정도의책임을갖는다.', metadata={'author': 'na', 'creationDate': D:20240103170645+09'00', 'creator': 'Microsoft® Word Microsoft 365용', 'file_path': '23-24풋살경기규칙.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': D:20240103170645+09'00', 'page': 33, 'producer': 'Microsoft® Word Microsoft 365용', 'source': '23-24풋살경기규칙.pdf', 'subject': '', 'title': '', 'total_pages': 175, 'trapped': ''})]'\
            question : 풋살 퇴장은 언제당해?",  #
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=300,
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정.
    )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.


### Instruction:
제시하는 context안에서만 대답해.            
context : [Document(page_content='146해석과권장사항규칙3 –선수들주심들의허가로피치밖에있는선수가,교체를하지않고,주심들또는3주심으로부터허가없이피치에재입장하여, 또다른경고성반칙을범했다면, 두개의경고성반칙에의해그선수를퇴장시켜야한다,예를들어, 주심들중한명의허가없이입장하여무모한태도로상대편을거는경우.이러한반칙이과도한힘으로범했다면, 그선수는직접퇴장이다.선수가피치의경계선중하나를우연히넘어가거나, 플레이움직임의일부로피치를떠난선수는반칙을범했다고간주되지않는다. 교체선수교체선수가교체절차를위반하여피치에입장하거나또는그팀의추가선수로플레이를하게되었다면, 주심들은주심외심판들의도움으로,다음의지침에따라야한다:
•플레이중단, 어드밴티지를적용할수있다면, 즉시중지시킬필요는없다.
•그팀의추가선수로플레이를한다면반스포츠적행동또는올바르지않게교체가이루어졌다면교체절차위반에의해경고를주어야한다.
•교체절차를완료하지않고입장하였거나또는그의팀의추가선수로플레이한이유의반칙이었고,그들이기술지역으로이동하기전이었다면,피치를떠나지않은교체선수는다음중단시에피치를떠나야한다.
•주심들이어드밴티지를적용했다면:  
•교체한팀이볼을소유했을때플레이를중단하여야하고볼이페널티에어리어안에있을때플레이가중단된경우를제외하고플레이가중단됐을때볼이있던위치에서상대팀의간접프리킥으로재개한다. (참고, 규칙13)
•교체한팀이간접프리킥,직접프리킥또는페널티킥으로처벌되는반칙또는교체선수가플레이를방해한경우, 상대팀에게관련된재개를주어교체선수팀을처벌하여야한다   필요하다면, 주심들역시범해진반칙에상응하는징계조치를한다.
•교체한팀의상대팀이반칙을범하여볼이아웃오브플레이가되는이유로플레이를중단한다음,   교체한팀의상대팀에게간접프리킥으로재개하여야한다.   필요하다면, 주심들역시범해

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
#model.save_pretrained("경로")  # 모델을 로컬에 저장합니다.
model.push_to_hub("Dongwookss/big_fut_final", token = hf_token)

### 옵션 1) 로컬에 저장

In [ ]:
base_model = "HuggingFaceH4/zephyr-7b-beta"
huggingface_token = huggingface_token
huggingface_repo = "Dongwookss/big_fut_final" # 모델을 업로드할 repository
save_method = "merged_16bit"              # "merged_4bit", "merged_4bit_forced", "merged_16bit", "lora"


In [ ]:
# 로컬 저장
model.save_pretrained_merged(
    base_model,
    tokenizer,
    save_method=save_method,  # 저장 방식을 16비트 병합으로 설정
)

In [ ]:
# Hub 에 업로드
model.push_to_hub_merged(
    huggingface_repo,
    tokenizer,
    save_method=save_method,
    token=huggingface_token,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 30.94 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:42<00:00,  1.33s/it]


Unsloth: Saving tokenizer...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

 Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


...

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/Dongwookss/big_fut_final


### GGUF 변환

Unsloth 는 `llama.cpp`를 복제하고 기본적으로 `q8_0`에 저장합니다.



### 옵션1) 로컬 저장

In [ ]:
quantization_method = "q8_0" # "f16" "q8_0" "q4_k_m" "q5_k_m"
model.save_pretrained_gguf(
    "path",
    tokenizer=tokenizer,
    quantization_method=quantization_method,
)

### 옵션2) HuggingFace 허브에 업로드

In [ ]:
# Quantization 방식 설정
quantization_method = "f16" # "f16" "q8_0" "q4_k_m" "q5_k_m"
# Hub 에 GGUF 업로드
model.push_to_hub_gguf(
    huggingface_repo + "-gguf",
    tokenizer,
    quantization_method=quantization_method,
    token=huggingface_token,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 35.12 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:25<00:00,  1.26it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at zephyr_tuning-gguf into f16 GGUF format.
The output location will be ./zephyr_tuning-gguf-unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: zephyr_tuning-gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 32768
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-to-gguf:gguf: rope theta = 10000.0
INFO